In [24]:
import pandas as pd
from langdetect import detect
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import numpy as np
import sys
import os
import time
from nltk.tag import pos_tag


In [2]:
target = pd.read_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1_Targets.csv",
                    sep = ',',
                     skiprows=range(112,113), # 112 wadliwy wiersz
                    names=['TableID', 'ColumnID', 'RowID'])
target[:3]

,TableID,ColumnID,RowID
0,50245608_0_871275842592178099,0,154
1,39107734_2_2329160387535788734,1,32
2,22864497_0_8632623712684511496,0,227


In [49]:
pos_tag("TimeShift".split())
detect("TimeShift")

'sq'

In [51]:
start = time.time()

answers_dict = {"TableID":[],
               "ColumnID":[],
               "RowID":[],
               "URI":[]}

counter = 0
errors = 0
for index, row in target.iterrows():
    try:
        counter = counter + 1
        if counter == 100:
            break


        table_id = row['TableID']
        column_id = row['ColumnID']
        row_id = row['RowID']

        df = pd.read_csv(f"D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1\{table_id}.csv",
                    sep=',')
        cell_content = df.iloc[row_id-1:row_id, column_id:(column_id+1)].values[0][0]
        cell_content = cell_content.replace('-', ' ')
        cell_content = cell_content.replace('The', '')
        cell_content = cell_content.replace(',', '')
        cell_content = cell_content.strip()
#         cell_content.replace('_', ' ')
        tags = pos_tag(cell_content.split())
        is_one_NNP = False
        for tag in tags:
            if tag[1] == "NNP":
                is_one_NNP = True
                break
        
        try:
            if is_one_NNP:
                lang = '@en'
            else:
                lang = str('@') + str(detect(cell_content))
        except Exception:
            lang = ''
        
        query = """SELECT ?resource WHERE { ?resource rdfs:label """ + '"' + str(cell_content) + '"' + "@en" + '}'
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        # sprawdzenie czy w ogóle znaleziono jakikolwiek link do dbpedii
        if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
            answers_dict['TableID'].append(table_id)
            answers_dict['ColumnID'].append(column_id)
            answers_dict['RowID'].append(row_id)
            answers_dict['URI'].append("BRAK")
            print(table_id, column_id, row_id)
            print(cell_content)
            print(query)
            errors = errors + 1
            continue

        # jeśli tak to szukamy konkretnie tego linku
        results_list = results['results']['bindings']
        for result_dict in results_list:
    #         print(result_dict)
            uri = result_dict['resource']['value']
            match = re.match("http://dbpedia.org/resource/*", uri)
            if match:
                answers_dict['TableID'].append(table_id)
                answers_dict['ColumnID'].append(column_id)
                answers_dict['RowID'].append(row_id)
                answers_dict['URI'].append(uri)
                break
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(table_id, column_id, row_id)
        print(cell_content)
        print(e)
        print(exc_tb.tb_lineno)
        continue



# print(answers_dict)
result_df = pd.DataFrame(answers_dict)
end = time.time()
print(end-start)
print(errors)

50270082_0_444360818941411589 1 119
Dazzy Vance?
SELECT ?resource WHERE { ?resource rdfs:label "Dazzy Vance?"@en}
41480166_0_6681239260286218499 0 177
Peter of Sebaste[?]
SELECT ?resource WHERE { ?resource rdfs:label "Peter of Sebaste[?]"@en}
91959037_0_7907661684242014480 1 22
Super Mario All Stars + Super Mario World
SELECT ?resource WHERE { ?resource rdfs:label "Super Mario All Stars + Super Mario World"@en}
13719111_1_5719401842463579519 0 48
Ruppell's griffon vulture
SELECT ?resource WHERE { ?resource rdfs:label "Ruppell's griffon vulture"@en}
90196673_0_5458330029110291950 0 190
TREE SWALLOW
SELECT ?resource WHERE { ?resource rdfs:label "TREE SWALLOW"@en}
54719588_0_8417197176086756912 3 195
Life as A House
SELECT ?resource WHERE { ?resource rdfs:label "Life as A House"@en}
90196673_0_5458330029110291950 0 310
ORCHARD ORIOLE
SELECT ?resource WHERE { ?resource rdfs:label "ORCHARD ORIOLE"@en}
91959037_0_7907661684242014480 1 303
Die Sims
SELECT ?resource WHERE { ?resource rdfs:labe

In [14]:
result_df

,TableID,ColumnID,RowID,URI
0,50245608_0_871275842592178099,0,154,http://dbpedia.org/resource/Pearl_Harbor
1,39107734_2_2329160387535788734,1,32,http://dbpedia.org/resource/Minneapolis-Saint_...
2,22864497_0_8632623712684511496,0,227,http://dbpedia.org/resource/Need_for_Speed:_Ca...
3,66009064_0_9148652238372261251,0,15,BRAK
4,21362676_0_6854186738074119688,1,75,BRAK
5,50245608_0_871275842592178099,0,186,BRAK
6,40534006_0_4617468856744635526,1,1,http://dbpedia.org/resource/Lex_Luger
7,36102169_0_7739454799295072814,2,286,http://dbpedia.org/resource/Crash_Bash
8,53822652_0_5767892317858575530,1,141,http://dbpedia.org/resource/The_Hurt_Locker
9,60319454_0_3938426910282115527,0,29,http://dbpedia.org/resource/Lion_(disambiguation)


In [ ]:
result_df.to_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\my_submission.csv",
                 index=False,
                 sep=',',
                header=False)

In [ ]:
df = pd.read_csv("D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\CEA_Round1\\13719111_1_5719401842463579519.csv",
                sep=',')
df[:3]

In [ ]:
detect(df.iloc[0:1, 1:2].values[0][0])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Norway> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
for result in results["results"]["bindings"]:
    print(result["label"]["value"])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ont: <http://dbpedia.org/ontology/>
    SELECT ?a, ?c
    WHERE { 
            ?a ont:location <http://dbpedia.org/resource/Delft> . 
            FILTER regex(?a, "Ikea", "i")
          }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
# print(result['a'])
# print(result['b'])
# print(result['c'])

# for result in results["results"]["bindings"]:
#     print(type(result))

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {e}
    VALUES (?r) {e} ({source}) {w}
        {e} ?r ?p ?o {w}
        UNION
        {e} ?s ?r ?o {w}
        UNION
        {e} ?s ?p ?r {w}
    {w} 
'''.format(source=sss, e="{", w="}"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {
    VALUES (?r) { (dbr:Gra_o_tron) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
    } 
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)



In [ ]:
SELECT ?r ?p ?o
WHERE {
 VALUES (?r) { (dbr:Cat) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
}

select ?s
where {
        { ?s rdfs:label "Kot domowy"@pl }
        UNION
        { "Kot domowy"@pl rdfs:label ?s }
} 

SELECT ?r
WHERE {
        { ?r rdfs:label "Kaspisches Meer"@de }

}

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?resource
    WHERE { ?resource rdfs:label "Kaspisches Meer"@de }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# print(results['results']['bindings'][1]['r']['value'])
# results = str(results['results']['bindings'])
print(str(results['results']['bindings']))
if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
    print("nie ma")
else:
    print('jest')
# for result_dict in results_list:
#     uri = result_dict['resource']['value']
#     match = re.match("http://dbpedia.org/resource/*", uri)
#     if match:
#         print(uri)
#     else:
#         print("NIE")

# print(results['results']['bindings'])

In [ ]:
import re
match = re.match("http://dbpedia.org/resource/*",'http://dbpedia.org/resource/Caspian_Sea', re.M|re.I)
if match:
    print(match.group())